# Projet 4

In [1]:
import requests
from bs4 import BeautifulSoup

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [2]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

# On importe la librairie python pour les expressions régulières
import re

def get_prices_from_url(url):
    
    request = requests.get(url)

    # On crée l'objet bs4
    soup = BeautifulSoup(request.content)

    # on initialise le dictionnaire "prices" 
    prices = {}

    # On récupère le numéro de la page html 
    num_page = url[32]
    num_page = int(num_page) # on transforme la variable num_page en entier
    #print(type(num_page))

    # On sélectionne toutes les divs renfermant toutes les autres divs contenant les informations que l'on souhaite scrapper
    soup2 = soup.findAll('div', attrs={'class': f'pure-u-1 pure-u-md-1-{num_page+1}'})

    #print(soup2)

    # On parcourt toutes les divs contenu dans la variable "soup2"
    for div in soup2:

        # On initialise le dictionnaire qui va contenir les différents prix associé à chaque catégorie de prix (i.e {Personal, Small Business,...})
        sub_dic_prices = {}

        # on sélectionne la div contenant les infos que l'on souhaite scraper
        soup3 = div.find('div', attrs = {'class': f'pricing-table-header'})

        # On récupère la catégorie de prix
        h2_balise = soup3.find('h2').text
        prices[h2_balise] = sub_dic_prices

        # On récupère le prix
        span_balise = soup3.find('span', attrs = {'class': 'pricing-table-price'})
        span_balise_pricing = span_balise.text[-14:-11] # on récupère uniquement la valeur numérique
        sub_dic_prices['price'] = span_balise_pricing.strip() # on supprime les blancs dans la chaine de caractères avec la fonction (strip)

        # On récupére le 3ème et le 4ème élément de la balise <li> qui sont eux mêmes renfermés dans une balise <ul>
        soup4 = div.find('ul', attrs = {'class': 'pricing-table-list'}).findAll('li')
        li_3_balise_storage = soup4[3].text.split()[0] # on ne garde que le pattern "xxxGB"
        sub_dic_prices['storage'] = li_3_balise_storage

        li_4_balise_databases = soup4[4].text.split()[0] # on ne garde uniquement le chiffre
        sub_dic_prices['databases'] = int(li_4_balise_databases) # on tranforme la valeur en entier
    

    return prices

get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [3]:
def extract_beer_infos(url):
    
    request = requests.get(url)

    # On crée l'objet bs4
    soup = BeautifulSoup(request.content)

    infos = {
        'name': None,
        'note': None,
        'price': None,
        'volume': None,
    }

    # On scrappe le "name"
    div_name = soup.find('div', attrs = {'class': f'product-detail-info-title'})
    h1_balise_name = div_name.find('h1').text
    infos["name"] = h1_balise_name

    # On scrappe la "note"
    data_div_note = soup.select('div[data-percent]')
    infos["note"] = int(data_div_note[0]['data-percent'])

    # On scrappe le "price"
    span_balise_price = soup.find('span', attrs = {'class': f'price'}).text
    span_balise_price_clean = span_balise_price[:-2].replace(",", ".")
    span_balice_price_float = float(span_balise_price_clean)
    infos["price"] = span_balice_price_float

    # On scrappe le "volume"
    div_balise_volume = soup.find('div', attrs = {'class': f'product-subtext'})
    span_balise_volume = div_balise_volume.find('span').text[-6:]
    span_balise_volume_cl = span_balise_volume.strip(' cl\n')
    infos["volume"] = int(span_balise_volume_cl)

    return infos

url = "https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33"
extract_beer_infos(url)

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [4]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [5]:
from multiprocessing import Pool

def extract_beer_list_infos(url):
    
    response = requests.get(url)
    data_beers = response.json()
    
    # On Collecte les pages de bières à partir de l'API json
    beer_pages = ['https://www.beerwulf.com' + item['contentReference'] for item in data_beers['items']]
    
    # Sequential version (slow):
    beers = [extract_beer_infos(url) for url in beer_pages]
    

    # Parallel version (faster):
    #p = Pool()
    #beers = p.map(extract_beer_infos,beer_pages)
    return beers

extract_beer_list_infos(URL_BEERLIST_FRANCE)

[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': 60,
  'price': 38.99,
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': 70,
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
 {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
 {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]

In [6]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 5.323s

OK
